In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_11.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    return max_corr if max_corr >= threshold else 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename):
    submatrix = pd.DataFrame(index=columns, columns=columns)
    for col1, col2 in product(columns, repeat=2):
        submatrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    submatrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "11_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub)

filename_cross = "11_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix11_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.219139,0.395456,0.115372,0.485857,0.625381,0.778111,0.706233,0.521172,0.690919,0.692623,...,0.173713,0.564549,0.485326,0.546072,0.090708,0.294336,0.653432,0.52905,0.479992,0.132007
21085.0,0.155637,0.359697,0.128991,0.375583,0.527051,0.596404,0.537467,0.506319,0.567823,0.454597,...,0.180812,0.403197,0.349567,0.423537,0.155848,0.247454,0.484427,0.481609,0.505363,0.177837
21087.0,0.054888,0.29824,0.142997,0.398053,0.562209,0.506782,0.484256,0.483544,0.565517,0.518089,...,0.136985,0.378211,0.48524,0.492124,0,0.340754,0.420933,0.465415,0.352501,0.209748
21089.0,0.281592,0.34844,0.179451,0.476977,0.553219,0.595445,0.555128,0.495815,0.562798,0.567029,...,0.403155,0.50093,0.435418,0.443681,0.126858,0.269139,0.506673,0.465386,0.542626,0.204086
21091.0,0.370445,0.459018,0.055691,0.500251,0.679046,0.773619,0.661817,0.504162,0.657609,0.612227,...,0.141415,0.54274,0.380056,0.52577,0.150328,0.278765,0.615419,0.515948,0.478941,0.158301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.255338,0.292379,0.18424,0.313278,0.168716,0.160445,0.109879,0.245598,0.136477,0.227497,...,0.267763,0.157463,0.109508,0.129543,0.216711,0.216386,0.13559,0.313653,0.202211,0.229575
38091.0,0.227859,0.245518,0.211974,0.281994,0.193189,0.129494,0.131023,0.205018,0.182242,0.155262,...,0.197351,0.184019,0.232288,0.21724,0.24972,0.305552,0.206822,0.159455,0.150612,0.233615
38093.0,0.42211,0.283473,0.227002,0.382243,0.095577,0.08802,0.055192,0,0.061874,0.248827,...,0.304105,0.140121,0,0.228329,0.62826,0.26107,0.076885,0.236009,0.164962,0.261836
38095.0,0.201291,0.279899,0.143685,0.235801,0.413613,0.298148,0.270001,0.152924,0.28581,0.25908,...,0.410182,0.358346,0.196177,0.25927,0.240201,0.141815,0.260868,0.360098,0.351028,0.213321
